In [40]:
import pickle
import neuromancer as nm
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.autograd.profiler as profiler

In [41]:
filepath = "QP_data/original/random_simple_dataset_var100_ineq50_eq50_ex10000"

with open(filepath, "rb") as f:
    data = pickle.load(f)



In [42]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, eq_cm, ineq_cm, eq_rhs, ineq_rhs):
        self.x = x
        self.eq_cm = eq_cm 
        self.ineq_cm = ineq_cm
        self.eq_rhs = eq_rhs
        self.ineq_rhs = ineq_rhs
        self._index = 0  # Internal index for tracking iteration

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # Return a tuple of input and target for the given index
        #! Change per data set.
        # return self.x[idx], self.eq_cm[idx], self.ineq_cm[idx], self.eq_rhs[idx], self.ineq_rhs[idx]
        return self.x[idx]

In [57]:
DTYPE = torch.float64
DEVICE = torch.device="cpu"
batch_size=200

X = data.X
eq_cm = data.eq_cm
ineq_cm = data.ineq_cm
eq_rhs = data.eq_rhs
ineq_rhs = data.ineq_rhs

Q = data.Q
p = data.p

print(f"eq_cm.shape: {eq_cm.shape}")
print(f"ineq_cm.shape: {ineq_cm.shape}")
print(f"eq_rhs.shape: {eq_rhs.shape}")
print(f"ineq_rhs.shape: {ineq_rhs.shape}")
print(f"Q.shape: {Q.shape}")
print(f"p.shape: {p.shape}")

train = data.train_indices
valid = data.valid_indices
test = data.test_indices

# Traning data in a data set
#! Vary per experiment

print(eq_rhs[data.train_indices].shape)

samples_train = {"eq_rhs": eq_rhs[data.train_indices]}
samples_valid = {"eq_rhs": eq_rhs[data.valid_indices]}
samples_test = {"eq_rhs": eq_rhs[data.test_indices]}

train_data = nm.dataset.DictDataset(samples_train, name='train')
valid_data = nm.dataset.DictDataset(samples_valid, name='dev')
test_data = nm.dataset.DictDataset(samples_test, name='test')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           collate_fn=train_data.collate_fn, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,
                                         collate_fn=valid_data.collate_fn, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         collate_fn=test_data.collate_fn, shuffle=True)

print(len(train_data))

func = nm.blocks.MLP(insize=data.xdim, outsize=data.ydim, bias=True, linear_map=nm.slim.maps['linear'], nonlin=nn.ReLU, hsizes=[500, 500])

sol_map = nm.Node(func, input_keys=['eq_rhs'], output_keys=['y'], name='map')

y = nm.variable("y")

eq_rhs = nm.variable("eq_rhs")

#1/2 * y^T Q y + p^Ty
# f = 0.5 * y @ Q @ y.T + p @ y
f = torch.sum(0.5 * (y @ Q) * y +  p * y, dim=1)
obj = f.minimize(weight=1.0, name="obj")

# eq_constraints = eq_cm @ y - eq_rhs == 0
eq_constraints = y @ eq_cm.T - eq_rhs == 0
ineq_constraints = y @ ineq_cm.T - ineq_rhs.view(1, -1) <= 0  # Broadcast ineq_rhs
# ineq_constraints = ineq_cm @ y - ineq_rhs <= 0

eq_constraints.name = 'eq_constraints'
ineq_constraints.name = 'ineq_constraints'

objectives = [obj]
constraints = [eq_constraints, ineq_constraints]
components = [sol_map]

loss = nm.PenaltyLoss(objectives, constraints)

problem = nm.Problem(components, loss)

lr = 0.001      # step size for gradient descent
epochs = 400    # number of training epochs
warmup = 100    # number of epochs to wait before enacting early stopping policy
patience = 100  # number of epochs with no improvement in eval metric to allow before early stopping

optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)

# define trainer
trainer = nm.Trainer(
    problem,
    train_loader,
    valid_loader,
    test_loader,
    optimizer,
    epochs=epochs,
    patience=patience,
    warmup=warmup)

# Train NLP solution map
best_model = trainer.train()
best_outputs = trainer.test(best_model)
# load best model dict
problem.load_state_dict(best_model)



eq_cm.shape: torch.Size([50, 100])
ineq_cm.shape: torch.Size([50, 100])
eq_rhs.shape: torch.Size([10000, 50])
ineq_rhs.shape: torch.Size([50])
Q.shape: torch.Size([100, 100])
p.shape: torch.Size([100])
torch.Size([8000, 50])
8000


/Users/benjacobs/miniconda3/lib/python3.10/site-packages/neuromancer/constraint.py:169: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([200, 50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(left, right)


epoch: 0  train_loss: -36.84498300780661
epoch: 1  train_loss: -50.47986237821413
epoch: 2  train_loss: -50.67859629690764
epoch: 3  train_loss: -50.68386467574614
epoch: 4  train_loss: -50.69908108232159
epoch: 5  train_loss: -50.70000986354354
epoch: 6  train_loss: -50.69434455754058
epoch: 7  train_loss: -50.72722819633897
epoch: 8  train_loss: -50.740797609557404
epoch: 9  train_loss: -50.722500858065956
epoch: 10  train_loss: -50.74292066495661
epoch: 11  train_loss: -50.737590612690894
epoch: 12  train_loss: -50.78030461830026
epoch: 13  train_loss: -50.76081135564972
epoch: 14  train_loss: -50.750392788929425
epoch: 15  train_loss: -50.79590942025992
epoch: 16  train_loss: -50.78301074716298
epoch: 17  train_loss: -50.774992834006625
epoch: 18  train_loss: -50.80526810826696
epoch: 19  train_loss: -50.78743292209125
epoch: 20  train_loss: -50.81181337292083
epoch: 21  train_loss: -50.79623466320928
epoch: 22  train_loss: -50.81260035213718
epoch: 23  train_loss: -50.805973393851

<All keys matched successfully>

In [67]:
for test_data in test_loader:
    model_out = problem(test_data)

data.obj_fn(model_out['test_y'])

sub


/Users/benjacobs/miniconda3/lib/python3.10/site-packages/neuromancer/constraint.py:169: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([200, 50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(left, right)


RuntimeError: The size of tensor a (100) must match the size of tensor b (50) at non-singleton dimension 1

RuntimeError: The size of tensor a (100) must match the size of tensor b (50) at non-singleton dimension 1